In [1]:
import numpy as np
import pandas as pd

import gc
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.layers import *
from tensorflow.python.keras.layers import Layer
from tensorflow.keras import regularizers

from tensorflow.keras.models import Model,load_model,Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

from tensorflow.keras import optimizers,initializers
from tensorflow.python.keras.initializers import glorot_normal


from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

C:\Users\Yunqing\.conda\envs\tfgpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Yunqing\.conda\envs\tfgpu\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Yunqing\.conda\envs\tfgpu\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


https://github.com/ShowMeAI-Hub/multi-task-learning/blob/main/README.md

In [2]:
data_origin = pd.read_excel('D:/OneDrive - University of South Carolina/Research/multitasking learning/data cleaning/DHEC_tests_final_lag1.xlsx', sheet_name='Sheet1', engine='openpyxl')
variables = pd.read_excel("D:/OneDrive - University of South Carolina/Research/multitasking learning/data cleaning/0130variables-VSLTCRIC.xlsx", sheet_name='Sheet1', engine='openpyxl')

## Model 1

In [3]:
temp = variables[variables['model1'] != 'delete']
data = data_origin[temp['variables'].tolist()]
data

,TIME_DXDATE_TESTDATE,time_window_index_year,VS,dx_yr,dx_mth,age,sex,race,risk,region,CD4_baseline,VL_baseline_interpretation,VL_baseline,linkage,retention
0,591.000000,2,0,2005,12,39,F,Black,Others,Urban,401.0,=,160288.0,0,0
1,1018.000000,3,0,2005,12,39,F,Black,Others,Urban,401.0,=,160288.0,0,0
2,2072.250000,6,1,2005,12,39,F,Black,Others,Urban,401.0,=,160288.0,0,1
3,2495.333333,7,1,2005,12,39,F,Black,Others,Urban,401.0,=,160288.0,0,1
4,2793.333333,8,1,2005,12,39,F,Black,Others,Urban,401.0,=,160288.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40274,2737.500000,8,1,2009,9,48,M,Black,MSM,Urban,414.0,=,15127.0,1,1
40275,3193.000000,9,1,2009,9,48,M,Black,MSM,Urban,414.0,=,15127.0,1,1
40276,3499.666667,10,1,2009,9,48,M,Black,MSM,Urban,414.0,=,15127.0,1,1
40277,3884.000000,11,1,2009,9,48,M,Black,MSM,Urban,414.0,=,15127.0,1,1


In [4]:
target = variables[variables['model1'] == 'outcome']['variables'].tolist()
sparse_features = variables[variables['model1'] == 'cat']['variables'].tolist()
dense_features = variables[variables['model1'] == 'num']['variables'].tolist()
varlen_features = []

In [5]:
sparse_features

['sex', 'race', 'risk', 'region', 'VL_baseline_interpretation']

In [6]:
encoder = {}
# 稀疏特征编码
for featid in sparse_features:
    # print(f"编码ID字段：{featid}")
    encoder[featid] = {uid:ucode+1 for ucode,uid in enumerate(data[featid].unique())} 
    data[featid] = data[featid].apply(lambda x: encoder[featid].get(x,0))
    
# 生成输入特征设置
sparse_max_len = {f:len(encoder[f]) + 1 for f in sparse_features}
varlens_max_len = {f:len(encoder[f]) + 1 for f in varlen_features}
feature_names = sparse_features+varlen_features+dense_features

C:\Users\Yunqing\AppData\Local\Temp\ipykernel_46932\2408031544.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[featid] = data[featid].apply(lambda x: encoder[featid].get(x,0))


In [7]:
n_train = round(data.shape[0] * 0.6)
n_val = round(data.shape[0] * 0.2)

train = data[:n_train]
val = data[n_train:(n_train+n_val)]
test = data[(n_train+n_val):]

In [8]:
# 构建输入数据
train_model_input = {name: train[name] if name not in varlen_features else np.stack(train[name]) for name in feature_names } #训练模型的输入，字典类型。名称和具体值
val_model_input = {name: val[name] if name not in varlen_features else np.stack(val[name]) for name in feature_names }
test_model_input = {name: test[name] if name not in varlen_features else np.stack(test[name]) for name in feature_names}

train_labels = [train[y].values for y in target]
val_labels = [val[y].values for y in target]
test_labels = [test[y].values for y in target]

### Seperate models

#### VS

In [11]:
X_train = data[:(n_train+n_val)].copy()
X_train.drop(columns=target, inplace=True)
y_train = data[:(n_train+n_val)]["VS"]
X_train

,TIME_DXDATE_TESTDATE,time_window_index_year,dx_yr,dx_mth,age,sex,race,risk,region,CD4_baseline,VL_baseline_interpretation,VL_baseline
0,591.000000,2,2005,12,39,1,1,1,1,401.0,1,160288.0
1,1018.000000,3,2005,12,39,1,1,1,1,401.0,1,160288.0
2,2072.250000,6,2005,12,39,1,1,1,1,401.0,1,160288.0
3,2495.333333,7,2005,12,39,1,1,1,1,401.0,1,160288.0
4,2793.333333,8,2005,12,39,1,1,1,1,401.0,1,160288.0
...,...,...,...,...,...,...,...,...,...,...,...,...
32218,1693.000000,5,2009,4,47,2,1,3,1,94.0,1,41899.0
32219,1964.000000,6,2009,4,47,2,1,3,1,94.0,1,41899.0
32220,2392.500000,7,2009,4,47,2,1,3,1,94.0,1,41899.0
32221,2771.000000,8,2009,4,47,2,1,3,1,94.0,1,41899.0


In [12]:
def check_class_distribution(y):
    class_counts = pd.Series(y).value_counts(normalize=True)  # 计算比例
    print(class_counts)

In [13]:
check_class_distribution(y_train)

1    0.714893
0    0.285107
Name: VS, dtype: float64


In [10]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                832       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 17,473
Trainable params: 17,473
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Train the model
history = model.fit(X_train.to_numpy(), np.array(y_train), epochs=20, batch_size=1000, validation_split=0.2)

Epoch 1/20
26/26 [==============================] - 2s 14ms/step - loss: 3643.7812 - accuracy: 0.6196 - val_loss: 32801.3086 - val_accuracy: 0.6580
Epoch 2/20
26/26 [==============================] - 0s 6ms/step - loss: 1526.8846 - accuracy: 0.5564 - val_loss: 1408.1959 - val_accuracy: 0.6687
Epoch 3/20
26/26 [==============================] - 0s 6ms/step - loss: 1555.8414 - accuracy: 0.5334 - val_loss: 7261.9946 - val_accuracy: 0.7544
Epoch 4/20
26/26 [==============================] - 0s 6ms/step - loss: 2713.4912 - accuracy: 0.6066 - val_loss: 7016.7007 - val_accuracy: 0.3170
Epoch 5/20
26/26 [==============================] - 0s 6ms/step - loss: 1990.4755 - accuracy: 0.5710 - val_loss: 4346.1470 - val_accuracy: 0.7465
Epoch 6/20
26/26 [==============================] - 0s 6ms/step - loss: 1016.3265 - accuracy: 0.5709 - val_loss: 22773.2168 - val_accuracy: 0.7131
Epoch 7/20
26/26 [==============================] - 0s 6ms/step - loss: 1399.0951 - accuracy: 0.5642 - val_loss: 2271.763

In [12]:
X_test = test.copy()
X_test.drop(columns=target, inplace=True)
pred = model.predict(X_test)
pred

array([[0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       ...,
       [1.9143881e-05],
       [1.3354896e-02],
       [1.2945627e-01]], dtype=float32)

In [13]:
print(accuracy_score(test_labels[0], (pred > 0.5).astype(int)))

0.304369414101291


In [14]:
print(roc_auc_score(test_labels[0], pred))

0.517228936949581


#### RIC

In [14]:
X_train = data[:(n_train+n_val)].copy()
X_train.drop(columns=target, inplace=True)
y_train = data[:(n_train+n_val)]["retention"]

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 训练逻辑回归模型
log_reg = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=42)
log_reg.fit(X_train.to_numpy(), np.array(y_train))

X_test = test.copy()
X_test.drop(columns=target, inplace=True)

# 在测试集上进行预测
log_pred_prob = log_reg.predict_proba(X_test)[:, 1]  # 获取正类的概率
log_pred = (log_pred_prob > 0.5).astype(int)  # 转换为二分类预测

# 计算 Accuracy 和 AUC
log_accuracy = accuracy_score(test_labels[0], log_pred)
log_auc = roc_auc_score(test_labels[0], log_pred_prob)

print(f"Logistic Regression Accuracy: {log_accuracy:.4f}")
print(f"Logistic Regression AUC: {log_auc:.4f}")

Logistic Regression Accuracy: 0.5675
Logistic Regression AUC: 0.4912


C:\Users\Yunqing\.conda\envs\tfgpu\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [22]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')   # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['AUC'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 17,473
Trainable params: 17,473
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Train the model
history = model.fit(X_train.to_numpy(), np.array(y_train), epochs=20, batch_size=1000, validation_split=0.2)

Epoch 1/20
26/26 [==============================] - 1s 12ms/step - loss: 4003.1680 - auc: 0.4969 - val_loss: 7143.0132 - val_auc: 0.4965
Epoch 2/20
26/26 [==============================] - 0s 6ms/step - loss: 1667.6602 - auc: 0.5013 - val_loss: 628.0534 - val_auc: 0.4999
Epoch 3/20
26/26 [==============================] - 0s 6ms/step - loss: 979.9077 - auc: 0.4947 - val_loss: 941.4921 - val_auc: 0.5001
Epoch 4/20
26/26 [==============================] - 0s 6ms/step - loss: 1054.4846 - auc: 0.5006 - val_loss: 12659.6709 - val_auc: 0.4879
Epoch 5/20
26/26 [==============================] - 0s 6ms/step - loss: 1024.0724 - auc: 0.4903 - val_loss: 20.3225 - val_auc: 0.5088
Epoch 6/20
26/26 [==============================] - 0s 5ms/step - loss: 694.7341 - auc: 0.5003 - val_loss: 240.8504 - val_auc: 0.4999
Epoch 7/20
26/26 [==============================] - 0s 5ms/step - loss: 1112.4059 - auc: 0.5065 - val_loss: 11549.0703 - val_auc: 0.4934
Epoch 8/20
26/26 [==============================] - 

In [24]:
X_test = test.copy()
X_test.drop(columns=target, inplace=True)
pred = model.predict(X_test)
pred

array([[1.        ],
       [1.        ],
       [1.        ],
       ...,
       [1.        ],
       [1.        ],
       [0.88537973]], dtype=float32)

In [25]:
print(accuracy_score(test_labels[1], (pred > 0.5).astype(int)))

0.7620407149950348


In [26]:
print(roc_auc_score(test_labels[1], pred))

0.5068592974793162


#### LTC

In [21]:
X_train = data[:(n_train+n_val)].copy()
X_train.drop(columns=target, inplace=True)
y_train = data[:(n_train+n_val)]["linkage"]

In [22]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')   # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_9 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 17,473
Trainable params: 17,473
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Train the model
history = model.fit(X_train.to_numpy(), np.array(y_train), epochs=20, batch_size=1000, validation_split=0.2)

Epoch 1/20
26/26 [==============================] - 1s 13ms/step - loss: 3892.3950 - accuracy: 0.6252 - val_loss: 81453.9844 - val_accuracy: 0.6925
Epoch 2/20
26/26 [==============================] - 0s 6ms/step - loss: 2405.5322 - accuracy: 0.5951 - val_loss: 16388.8887 - val_accuracy: 0.6276
Epoch 3/20
26/26 [==============================] - 0s 5ms/step - loss: 327.0363 - accuracy: 0.5823 - val_loss: 534.7205 - val_accuracy: 0.5452
Epoch 4/20
26/26 [==============================] - 0s 6ms/step - loss: 444.3952 - accuracy: 0.5949 - val_loss: 32937.6523 - val_accuracy: 0.6925
Epoch 5/20
26/26 [==============================] - 0s 5ms/step - loss: 808.4762 - accuracy: 0.5801 - val_loss: 2747.5989 - val_accuracy: 0.6441
Epoch 6/20
26/26 [==============================] - 0s 5ms/step - loss: 547.1274 - accuracy: 0.6132 - val_loss: 4451.8521 - val_accuracy: 0.5893
Epoch 7/20
26/26 [==============================] - 0s 6ms/step - loss: 403.2219 - accuracy: 0.6127 - val_loss: 20222.7344 - 

In [24]:
X_test = test.copy()
X_test.drop(columns=target, inplace=True)
pred = model.predict(X_test)
pred

array([[0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       ...,
       [4.7233419e-12],
       [2.0811015e-14],
       [1.0000000e+00]], dtype=float32)

In [25]:
print(accuracy_score(test_labels[1], (pred > 0.5).astype(int)))

0.6052631578947368


In [26]:
print(roc_auc_score(test_labels[1], pred))

0.5579107755259762


### MMoE

In [28]:
class MeanPoolLayer(Layer):
    def __init__(self, axis, **kwargs):
        super(MeanPoolLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, x, mask):
        mask = tf.expand_dims(tf.cast(mask,tf.float32),axis = -1)
        x = x * mask
        return K.sum(x, axis=self.axis) / (K.sum(mask, axis=self.axis) + 1e-9)

class MmoeLayer(tf.keras.layers.Layer):
    def __init__(self,expert_dim,n_expert,n_task):
        super(MmoeLayer, self).__init__()
        self.n_task = n_task
        self.expert_layer = [Dense(expert_dim,activation = 'relu') for i in range(n_expert)]
        self.gate_layers = [Dense(n_expert,activation = 'softmax') for i in range(n_task)]
    
    def call(self,x):
        #多个专家网络
        E_net = [expert(x) for expert in self.expert_layer]
        E_net = Concatenate(axis = 1)([e[:,tf.newaxis,:] for e in E_net]) #(bs,n_expert,n_dims)
        #多个门网络
        gate_net = [gate(x) for gate in self.gate_layers]     #n_task个(bs,n_expert)
        
        #每个towers等于，对应的门网络乘上所有的专家网络。
        towers = []
        for i in range(self.n_task):
            g = tf.expand_dims(gate_net[i],axis = -1)  #(bs,n_expert,1)
            _tower = tf.matmul(E_net, g,transpose_a=True)
            towers.append(Flatten()(_tower))           #(bs,expert_dim)
            
        return towers

def build_mmoe(sparse_cols,dense_cols,sparse_max_len,embed_dim,expert_dim,
              varlens_cols,varlens_max_len,n_expert,n_task,target = [],
              dnn_hidden_units = (64,),dnn_reg_l2 = 1e-5,drop_rate = 0.1,
                embedding_reg_l2 = 1e-6):
    
    
    #输入部分，分为sparse,varlens,dense部分。
    sparse_inputs = {f:Input([1],name = f) for f in sparse_cols}
    dense_inputs = {f:Input([1],name = f) for f in dense_cols}
    varlens_inputs = {f:Input([None,1],name = f) for f in varlens_cols}
        
    input_embed = {}
    #离散特征，embedding到k维
    for f in sparse_cols:
        _input = sparse_inputs[f]
        embedding = Embedding(sparse_max_len[f], embed_dim, 
            embeddings_regularizer=tf.keras.regularizers.l2(embedding_reg_l2)) 
        input_embed[f] =Flatten()(embedding(_input)) #(bs,k)
        
    #多标签离散变量
    for f in varlens_inputs:
        _input = varlens_inputs[f]
        mask = Masking(mask_value = 0).compute_mask(_input)
        embedding = Embedding(varlens_max_len[f], embed_dim,
            embeddings_regularizer=tf.keras.regularizers.l2(1e-6))
        _embed =Reshape([-1,embed_dim])(embedding(_input))
        out_embed = MeanPoolLayer(axis=1)(_embed,mask)
        input_embed[f] = out_embed
        
    input_embed.update(dense_inputs) #加入连续变量
    input_embed = Concatenate(axis = -1)([input_embed[f] for f in input_embed])
    for num in dnn_hidden_units:
        input_embed = Dropout(drop_rate)(Dense(num,activation = 'relu',
                    kernel_regularizer=regularizers.l2(dnn_reg_l2))(input_embed))
    
    #mmoe网络层
    towers = MmoeLayer(expert_dim,n_expert,n_task)(input_embed)
    outputs = [Dense(1,activation = 'sigmoid', kernel_regularizer=regularizers.l2(dnn_reg_l2),
                     name = f,use_bias = True)(_t) for _t,f in zip(towers,target)]
    inputs = [sparse_inputs[f] for f in sparse_inputs]+[varlens_inputs[f] for f in varlens_inputs]\
                +[dense_inputs[f] for f in dense_inputs]
    model = Model(inputs,outputs) 
    return model

In [29]:
# 构建模型，训练和评估
model = build_mmoe(sparse_features,dense_features,sparse_max_len,embed_dim = 32,expert_dim = 8,
          n_task = 3,n_expert = 4,varlens_cols = varlen_features,varlens_max_len = varlens_max_len,
          dnn_hidden_units = (32,64,32),target = target,dnn_reg_l2 = 0.001,drop_rate = 0.1)

adam = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(adam, loss = 'binary_crossentropy' ,metrics = ["accuracy"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sex (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
race (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
risk (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
region (InputLayer)             [(None, 1)]          0                                            
______________________________________________________________________________________________

In [30]:
history = model.fit(train_model_input, train_labels,validation_data = (val_model_input,val_labels),
                    batch_size=1000, epochs=20, verbose=1)

Epoch 1/20
25/25 [==============================] - 2s 30ms/step - loss: 3860.0076 - VS_loss: 1748.6101 - linkage_loss: 694.0479 - retention_loss: 1417.2137 - VS_accuracy: 0.5471 - linkage_accuracy: 0.5303 - retention_accuracy: 0.7132 - val_loss: 1408.4767 - val_VS_loss: 276.5470 - val_linkage_loss: 1004.9866 - val_retention_loss: 126.8167 - val_VS_accuracy: 0.7430 - val_linkage_accuracy: 0.6939 - val_retention_accuracy: 0.7663
Epoch 2/20
25/25 [==============================] - 0s 19ms/step - loss: 2257.4983 - VS_loss: 827.0262 - linkage_loss: 333.1529 - retention_loss: 1097.1987 - VS_accuracy: 0.5704 - linkage_accuracy: 0.5988 - retention_accuracy: 0.7310 - val_loss: 2746.1060 - val_VS_loss: 2134.8218 - val_linkage_loss: 568.9769 - val_retention_loss: 42.1928 - val_VS_accuracy: 0.7439 - val_linkage_accuracy: 0.6970 - val_retention_accuracy: 0.7612
Epoch 3/20
25/25 [==============================] - 0s 18ms/step - loss: 1915.1398 - VS_loss: 1081.3909 - linkage_loss: 198.5350 - retenti

25/25 [==============================] - 0s 15ms/step - loss: 18.5258 - VS_loss: 6.8651 - linkage_loss: 4.5576 - retention_loss: 7.0188 - VS_accuracy: 0.7004 - linkage_accuracy: 0.6960 - retention_accuracy: 0.8208 - val_loss: 1.8911 - val_VS_loss: 0.6121 - val_linkage_loss: 0.6490 - val_retention_loss: 0.5458 - val_VS_accuracy: 0.7442 - val_linkage_accuracy: 0.6964 - val_retention_accuracy: 0.8419


In [31]:
pred = model.predict(test_model_input)
pred

[array([[0.6041793],
        [0.6041793],
        [0.6041793],
        ...,
        [0.6041793],
        [0.6041793],
        [0.6041793]], dtype=float32),
 array([[0.5675649 ],
        [0.5675649 ],
        [0.5675649 ],
        ...,
        [0.5675649 ],
        [0.5675649 ],
        [0.65216166]], dtype=float32),
 array([[0.63046795],
        [0.63046795],
        [0.63046795],
        ...,
        [0.63046795],
        [0.63046795],
        [0.63046795]], dtype=float32)]

In [32]:
print(accuracy_score(test_labels[0], (pred[0] > 0.5).astype(int)))
print(accuracy_score(test_labels[1], (pred[1] > 0.5).astype(int)))
print(accuracy_score(test_labels[2], (pred[2] > 0.5).astype(int)))

0.7768123138033763
0.7671300893743793
0.8259682224428997


In [33]:
print(roc_auc_score(test_labels[0], pred[0]))
print(roc_auc_score(test_labels[1], pred[1]))
print(roc_auc_score(test_labels[2], pred[2]))

0.5
0.5052839995583801
0.5


### PLE

In [33]:
class MeanPoolLayer(Layer):
    def __init__(self, axis, **kwargs):
        super(MeanPoolLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, x, mask):
        mask = tf.expand_dims(tf.cast(mask,tf.float32),axis = -1)
        x = x * mask
        return K.sum(x, axis=self.axis) / (K.sum(mask, axis=self.axis) + 1e-9)

class PleLayer(tf.keras.layers.Layer):
    '''
    n_experts:list,每个任务使用几个expert。[2,3]第一个任务使用2个expert，第二个任务使用3个expert。
    n_expert_share:int,共享的部分设置的expert个数。
    expert_dim:int,每个专家网络输出的向量维度。
    n_task:int,任务个数。
    '''
    def __init__(self,n_task,n_experts,expert_dim,n_expert_share,dnn_reg_l2 = 1e-5):
        super(PleLayer, self).__init__()
        self.n_task = n_task
        
        # 生成多个任务特定网络和1个共享网络。
        self.E_layer = []
        for i in range(n_task):
            sub_exp = [Dense(expert_dim,activation = 'relu') for j in range(n_experts[i])]
            self.E_layer.append(sub_exp)
            
        self.share_layer = [Dense(expert_dim,activation = 'relu') for j in range(n_expert_share)]
        #定义门控网络
        self.gate_layers = [Dense(n_expert_share+n_experts[i],kernel_regularizer=regularizers.l2(dnn_reg_l2),
                                  activation = 'softmax') for i in range(n_task)]

    def call(self,x):
        #特定网络和共享网络
        E_net = [[expert(x) for expert in sub_expert] for sub_expert in self.E_layer]
        share_net = [expert(x) for expert in self.share_layer]
        
        #门的权重乘上，指定任务和共享任务的输出。
        towers = []
        for i in range(self.n_task):
            g = self.gate_layers[i](x)
            g = tf.expand_dims(g,axis = -1) #(bs,n_expert_share+n_experts[i],1)
            _e = share_net+E_net[i]  
            _e = Concatenate(axis = 1)([expert[:,tf.newaxis,:] for expert in _e]) #(bs,n_expert_share+n_experts[i],expert_dim)
            _tower = tf.matmul(_e, g,transpose_a=True)
            towers.append(Flatten()(_tower)) #(bs,expert_dim)
        return towers

def build_ple(sparse_cols,dense_cols,sparse_max_len,embed_dim,expert_dim = 4,
              varlens_cols = [],varlens_max_len = [],dnn_hidden_units = (64,64),
              n_task = 2,n_experts = [2,2],n_expert_share = 4,dnn_reg_l2 = 1e-6,
              drop_rate = 0.0,embedding_reg_l2 = 1e-6,targets = []):

   #输入部分，分为sparse,varlens,dense部分。
    sparse_inputs = {f:Input([1],name = f) for f in sparse_cols}
    dense_inputs = {f:Input([1],name = f) for f in dense_cols}
    varlens_inputs = {f:Input([None,1],name = f) for f in varlens_cols}
        
    input_embed = {}
    #离散特征，embedding到k维
    for f in sparse_cols:
        _input = sparse_inputs[f]
        embedding = Embedding(sparse_max_len[f], embed_dim, 
            embeddings_regularizer=tf.keras.regularizers.l2(embedding_reg_l2)) 
        input_embed[f] =Flatten()(embedding(_input)) #(bs,k)
        
    #多标签离散变量
    for f in varlens_inputs:
        _input = varlens_inputs[f]
        mask = Masking(mask_value = 0).compute_mask(_input)
        embedding = Embedding(varlens_max_len[f], embed_dim,
            embeddings_regularizer=tf.keras.regularizers.l2(1e-6))
        _embed =Reshape([-1,embed_dim])(embedding(_input))
        out_embed = MeanPoolLayer(axis=1)(_embed,mask)
        input_embed[f] = out_embed
        
    input_embed.update(dense_inputs) #加入连续变量
    input_embed = Concatenate(axis = -1)([input_embed[f] for f in input_embed])    
                                  
    for num in dnn_hidden_units:
        input_embed = Dropout(drop_rate)(Dense(num,activation = 'relu',
                    kernel_regularizer=regularizers.l2(dnn_reg_l2))(input_embed))
    #Ple网络层
    towers = PleLayer(n_task,n_experts,expert_dim,n_expert_share)(input_embed)
    outputs = [Dense(1,activation = 'sigmoid',kernel_regularizer=regularizers.l2(dnn_reg_l2),
                       name = f,use_bias = True)(_t) for f,_t in zip(targets,towers)]
    inputs = [sparse_inputs[f] for f in sparse_inputs]+[varlens_inputs[f] for f in varlens_inputs]\
                +[dense_inputs[f] for f in dense_inputs]
    model = Model(inputs,outputs) 
    return model

In [34]:
# 构建模型，训练和评估
model = build_ple(sparse_features,dense_features,sparse_max_len,embed_dim = 64,expert_dim = 16,
          varlens_cols = varlen_features,varlens_max_len = varlens_max_len,dnn_hidden_units = (64,64),
          n_task = 3,n_experts = [1,1,1],n_expert_share = 2,dnn_reg_l2 = 0.001,
          drop_rate = 0.1,embedding_reg_l2 = 0.001,targets = target)

adam = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(adam, loss = 'binary_crossentropy' ,metrics = ["accuracy"],)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sex (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
race (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
risk (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
region (InputLayer)             [(None, 1)]          0                                            
____________________________________________________________________________________________

In [35]:
history = model.fit(train_model_input, train_labels,validation_data = (val_model_input,val_labels),
                    batch_size=1000, epochs=20, verbose=1)

Epoch 1/20
25/25 [==============================] - 4s 48ms/step - loss: 5585.0957 - VS_loss: 3462.7629 - linkage_loss: 1191.8312 - retention_loss: 930.3781 - VS_accuracy: 0.5959 - linkage_accuracy: 0.5954 - retention_accuracy: 0.6851 - val_loss: 2412.0095 - val_VS_loss: 301.5742 - val_linkage_loss: 343.3718 - val_retention_loss: 1766.9329 - val_VS_accuracy: 0.7442 - val_linkage_accuracy: 0.3592 - val_retention_accuracy: 0.3292
Epoch 2/20
25/25 [==============================] - 1s 22ms/step - loss: 538.9778 - VS_loss: 255.0327 - linkage_loss: 74.0181 - retention_loss: 209.7876 - VS_accuracy: 0.5895 - linkage_accuracy: 0.5744 - retention_accuracy: 0.7627 - val_loss: 748.1414 - val_VS_loss: 746.7322 - val_linkage_loss: 0.6776 - val_retention_loss: 0.6059 - val_VS_accuracy: 0.7447 - val_linkage_accuracy: 0.6964 - val_retention_accuracy: 0.8419
Epoch 3/20
25/25 [==============================] - 1s 25ms/step - loss: 360.5919 - VS_loss: 353.9021 - linkage_loss: 2.6620 - retention_loss: 3.9

In [36]:
pred = model.predict(test_model_input)
pred

[array([[0.70601195],
        [0.70601195],
        [0.70601195],
        ...,
        [0.70601195],
        [0.70601195],
        [0.70601195]], dtype=float32),
 array([[0.6990726],
        [0.6990726],
        [0.6990726],
        ...,
        [0.6990726],
        [0.6990726],
        [0.6990726]], dtype=float32),
 array([[0.8311868],
        [0.8311868],
        [0.8311868],
        ...,
        [0.8311868],
        [0.8311868],
        [0.8311868]], dtype=float32)]

In [37]:
print(accuracy_score(test_labels[0], (pred[0] > 0.5).astype(int)))
print(accuracy_score(test_labels[1], (pred[1] > 0.5).astype(int)))
print(accuracy_score(test_labels[2], (pred[2] > 0.5).astype(int)))

0.7768123138033763
0.7671300893743793
0.8259682224428997


In [38]:
print(roc_auc_score(test_labels[0], pred[0]))
print(roc_auc_score(test_labels[1], pred[1]))
print(roc_auc_score(test_labels[2], pred[2]))

0.5
0.5
0.5


## Model 2

In [39]:
temp = variables[variables['model2'] != 'delete']
data = data_origin[temp['variables'].tolist()]
data

,TIME_DXDATE_TESTDATE,time_window_index_year,VS,dx_yr,dx_mth,age,sex,race,risk,region,...,msld.cum_1,metacanc.cum_1,aids.cum_1,Depression_1,Anxiety_1,Psychiatric_disorder_1,Alcohol_use_1,Tobacco_use_1,Illicit_drug_use_1,visits_1
0,591.000000,2,0,2005,12,39,F,Black,Others,Urban,...,0.0,0.0,0.0,0,0,0,0,0,0,4
1,1018.000000,3,0,2005,12,39,F,Black,Others,Urban,...,0.0,0.0,0.0,0,1,0,1,1,1,2
2,2072.250000,6,1,2005,12,39,F,Black,Others,Urban,...,0.0,0.0,0.0,0,0,0,0,0,0,2
3,2495.333333,7,1,2005,12,39,F,Black,Others,Urban,...,0.0,0.0,0.0,0,0,0,0,0,0,8
4,2793.333333,8,1,2005,12,39,F,Black,Others,Urban,...,0.0,0.0,0.0,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40274,2737.500000,8,1,2009,9,48,M,Black,MSM,Urban,...,0.0,0.0,0.0,0,0,0,0,0,0,3
40275,3193.000000,9,1,2009,9,48,M,Black,MSM,Urban,...,0.0,0.0,0.0,0,0,0,0,0,0,8
40276,3499.666667,10,1,2009,9,48,M,Black,MSM,Urban,...,0.0,0.0,0.0,0,0,0,0,0,0,3
40277,3884.000000,11,1,2009,9,48,M,Black,MSM,Urban,...,0.0,0.0,0.0,0,0,0,0,0,0,6


In [40]:
target = variables[variables['model2'] == 'outcome']['variables'].tolist()
sparse_features = variables[variables['model2'] == 'cat']['variables'].tolist()
dense_features = variables[variables['model2'] == 'num']['variables'].tolist()
varlen_features = []

In [41]:
sparse_features

['sex',
 'race',
 'risk',
 'region',
 'VL_baseline_interpretation',
 'mi',
 'chf',
 'pvd',
 'cevd',
 'dementia',
 'cpd',
 'rheumd',
 'pud',
 'mld',
 'diab',
 'diabwc',
 'hp',
 'rend',
 'canc',
 'msld',
 'metacanc',
 'aids',
 'Depression',
 'Anxiety',
 'Psychiatric_disorder',
 'Alcohol_use',
 'Tobacco_use',
 'Illicit_drug_use',
 'county',
 'mi_1',
 'chf_1',
 'pvd_1',
 'cevd_1',
 'dementia_1',
 'cpd_1',
 'rheumd_1',
 'pud_1',
 'mld_1',
 'diab_1',
 'diabwc_1',
 'hp_1',
 'rend_1',
 'canc_1',
 'msld_1',
 'metacanc_1',
 'aids_1',
 'Depression_1',
 'Anxiety_1',
 'Psychiatric_disorder_1',
 'Alcohol_use_1',
 'Tobacco_use_1',
 'Illicit_drug_use_1']

In [42]:
encoder = {}
# 稀疏特征编码
for featid in sparse_features:
    # print(f"编码ID字段：{featid}")
    encoder[featid] = {uid:ucode+1 for ucode,uid in enumerate(data[featid].unique())} 
    data[featid] = data[featid].apply(lambda x: encoder[featid].get(x,0))
    
# 生成输入特征设置
sparse_max_len = {f:len(encoder[f]) + 1 for f in sparse_features}
varlens_max_len = {f:len(encoder[f]) + 1 for f in varlen_features}
feature_names = sparse_features+varlen_features+dense_features

C:\Users\Yunqing\AppData\Local\Temp\ipykernel_20536\2408031544.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[featid] = data[featid].apply(lambda x: encoder[featid].get(x,0))


In [43]:
n_train = round(data.shape[0] * 0.6)
n_val = round(data.shape[0] * 0.2)

train = data[:n_train]
val = data[n_train:(n_train+n_val)]
test = data[(n_train+n_val):]

In [44]:
# 构建输入数据
train_model_input = {name: train[name] if name not in varlen_features else np.stack(train[name]) for name in feature_names } #训练模型的输入，字典类型。名称和具体值
val_model_input = {name: val[name] if name not in varlen_features else np.stack(val[name]) for name in feature_names }
test_model_input = {name: test[name] if name not in varlen_features else np.stack(test[name]) for name in feature_names}

train_labels = [train[y].values for y in target]
val_labels = [val[y].values for y in target]
test_labels = [test[y].values for y in target]

### Seperate models

#### VS

In [45]:
X_train = data[:(n_train+n_val)].copy()
X_train.drop(columns=target, inplace=True)
y_train = data[:(n_train+n_val)]["VS"]
X_train

,TIME_DXDATE_TESTDATE,time_window_index_year,dx_yr,dx_mth,age,sex,race,risk,region,CD4_baseline,...,msld.cum_1,metacanc.cum_1,aids.cum_1,Depression_1,Anxiety_1,Psychiatric_disorder_1,Alcohol_use_1,Tobacco_use_1,Illicit_drug_use_1,visits_1
0,591.000000,2,2005,12,39,1,1,1,1,401.0,...,0.0,0.0,0.0,1,1,1,1,1,1,4
1,1018.000000,3,2005,12,39,1,1,1,1,401.0,...,0.0,0.0,0.0,1,2,1,2,2,2,2
2,2072.250000,6,2005,12,39,1,1,1,1,401.0,...,0.0,0.0,0.0,1,1,1,1,1,1,2
3,2495.333333,7,2005,12,39,1,1,1,1,401.0,...,0.0,0.0,0.0,1,1,1,1,1,1,8
4,2793.333333,8,2005,12,39,1,1,1,1,401.0,...,0.0,0.0,0.0,1,1,1,1,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32218,1693.000000,5,2009,4,47,2,1,3,1,94.0,...,0.0,0.0,0.0,1,1,1,1,1,1,4
32219,1964.000000,6,2009,4,47,2,1,3,1,94.0,...,0.0,0.0,0.0,1,1,1,1,1,1,2
32220,2392.500000,7,2009,4,47,2,1,3,1,94.0,...,0.0,0.0,0.0,1,1,1,1,1,1,6
32221,2771.000000,8,2009,4,47,2,1,3,1,94.0,...,0.0,0.0,0.0,1,1,1,1,1,1,5


In [46]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 64)                6656      
_________________________________________________________________
dense_32 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_33 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 65        
Total params: 23,297
Trainable params: 23,297
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Train the model
history = model.fit(X_train.to_numpy(), np.array(y_train), epochs=20, batch_size=1000, validation_split=0.2)

Epoch 1/20
26/26 [==============================] - 1s 13ms/step - loss: 2015.4648 - accuracy: 0.5978 - val_loss: 2137.0535 - val_accuracy: 0.7528
Epoch 2/20
26/26 [==============================] - 0s 7ms/step - loss: 1129.7391 - accuracy: 0.5566 - val_loss: 2220.9998 - val_accuracy: 0.3466
Epoch 3/20
26/26 [==============================] - 0s 7ms/step - loss: 5029.7529 - accuracy: 0.5708 - val_loss: 3951.7371 - val_accuracy: 0.7544
Epoch 4/20
26/26 [==============================] - 0s 7ms/step - loss: 314.7221 - accuracy: 0.6536 - val_loss: 18052.6738 - val_accuracy: 0.7544
Epoch 5/20
26/26 [==============================] - 0s 6ms/step - loss: 2341.8276 - accuracy: 0.6059 - val_loss: 15401.9697 - val_accuracy: 0.7544
Epoch 6/20
26/26 [==============================] - 0s 7ms/step - loss: 756.3921 - accuracy: 0.6129 - val_loss: 450.1421 - val_accuracy: 0.2911
Epoch 7/20
26/26 [==============================] - 0s 7ms/step - loss: 1414.1932 - accuracy: 0.5719 - val_loss: 4207.1484 -

In [48]:
X_test = test.copy()
X_test.drop(columns=target, inplace=True)
pred = model.predict(X_test)
pred

array([[1.        ],
       [1.        ],
       [1.        ],
       ...,
       [0.6683879 ],
       [0.16565925],
       [0.7555089 ]], dtype=float32)

In [49]:
print(accuracy_score(test_labels[0], (pred > 0.5).astype(int)))

0.631454816285998


In [50]:
print(roc_auc_score(test_labels[0], pred))

0.5060499201733684


#### RIC

In [51]:
X_train = data[:(n_train+n_val)].copy()
X_train.drop(columns=target, inplace=True)
y_train = data[:(n_train+n_val)]["retention"]

In [52]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')   # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 64)                6656      
_________________________________________________________________
dense_36 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_37 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 65        
Total params: 23,297
Trainable params: 23,297
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Train the model
history = model.fit(X_train.to_numpy(), np.array(y_train), epochs=20, batch_size=1000, validation_split=0.2)

Epoch 1/20
26/26 [==============================] - 1s 14ms/step - loss: 4378.0781 - accuracy: 0.5598 - val_loss: 453.6558 - val_accuracy: 0.8295
Epoch 2/20
26/26 [==============================] - 0s 6ms/step - loss: 1236.2568 - accuracy: 0.7796 - val_loss: 47.4220 - val_accuracy: 0.6971
Epoch 3/20
26/26 [==============================] - 0s 7ms/step - loss: 251.5029 - accuracy: 0.7025 - val_loss: 26.1400 - val_accuracy: 0.7682
Epoch 4/20
26/26 [==============================] - 0s 8ms/step - loss: 326.8781 - accuracy: 0.7193 - val_loss: 236.3590 - val_accuracy: 0.8377
Epoch 5/20
26/26 [==============================] - 0s 6ms/step - loss: 1315.9252 - accuracy: 0.6128 - val_loss: 779.9474 - val_accuracy: 0.8379
Epoch 6/20
26/26 [==============================] - 0s 7ms/step - loss: 1658.0767 - accuracy: 0.6215 - val_loss: 828.1443 - val_accuracy: 0.8284
Epoch 7/20
26/26 [==============================] - 0s 7ms/step - loss: 1702.4854 - accuracy: 0.6828 - val_loss: 1283.3475 - val_accu

In [54]:
X_test = test.copy()
X_test.drop(columns=target, inplace=True)
pred = model.predict(X_test)
pred

array([[1.       ],
       [1.       ],
       [1.       ],
       ...,
       [1.       ],
       [1.       ],
       [0.9999392]], dtype=float32)

In [55]:
print(accuracy_score(test_labels[1], (pred > 0.5).astype(int)))

0.7637785501489573


In [56]:
print(roc_auc_score(test_labels[1], pred))

0.49640890554163997


#### LTC

In [57]:
X_train = data[:(n_train+n_val)].copy()
X_train.drop(columns=target, inplace=True)
y_train = data[:(n_train+n_val)]["linkage"]

In [58]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')   # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 64)                6656      
_________________________________________________________________
dense_40 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_41 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 65        
Total params: 23,297
Trainable params: 23,297
Non-trainable params: 0
_________________________________________________________________


In [59]:
# Train the model
history = model.fit(X_train.to_numpy(), np.array(y_train), epochs=20, batch_size=1000, validation_split=0.2)

Epoch 1/20
26/26 [==============================] - 1s 14ms/step - loss: 1354.9604 - accuracy: 0.6412 - val_loss: 25574.6777 - val_accuracy: 0.6340
Epoch 2/20
26/26 [==============================] - 0s 7ms/step - loss: 504.2840 - accuracy: 0.5647 - val_loss: 5545.3232 - val_accuracy: 0.6735
Epoch 3/20
26/26 [==============================] - 0s 6ms/step - loss: 287.9030 - accuracy: 0.6101 - val_loss: 5596.7671 - val_accuracy: 0.6940
Epoch 4/20
26/26 [==============================] - 0s 7ms/step - loss: 124.4110 - accuracy: 0.6001 - val_loss: 6061.0967 - val_accuracy: 0.6925
Epoch 5/20
26/26 [==============================] - 0s 7ms/step - loss: 183.7287 - accuracy: 0.6203 - val_loss: 5747.1182 - val_accuracy: 0.6506
Epoch 6/20
26/26 [==============================] - 0s 6ms/step - loss: 611.6110 - accuracy: 0.5752 - val_loss: 3852.7368 - val_accuracy: 0.6425
Epoch 7/20
26/26 [==============================] - 0s 6ms/step - loss: 200.5870 - accuracy: 0.6203 - val_loss: 95.4889 - val_a

In [60]:
X_test = test.copy()
X_test.drop(columns=target, inplace=True)
pred = model.predict(X_test)
pred

array([[0.       ],
       [0.       ],
       [0.       ],
       ...,
       [0.       ],
       [0.       ],
       [0.9427753]], dtype=float32)

In [61]:
print(accuracy_score(test_labels[1], (pred > 0.5).astype(int)))

0.5763406156901688


In [62]:
print(roc_auc_score(test_labels[1], pred))

0.5547145945032121


### MMoE

In [63]:
class MeanPoolLayer(Layer):
    def __init__(self, axis, **kwargs):
        super(MeanPoolLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, x, mask):
        mask = tf.expand_dims(tf.cast(mask,tf.float32),axis = -1)
        x = x * mask
        return K.sum(x, axis=self.axis) / (K.sum(mask, axis=self.axis) + 1e-9)

class MmoeLayer(tf.keras.layers.Layer):
    def __init__(self,expert_dim,n_expert,n_task):
        super(MmoeLayer, self).__init__()
        self.n_task = n_task
        self.expert_layer = [Dense(expert_dim,activation = 'relu') for i in range(n_expert)]
        self.gate_layers = [Dense(n_expert,activation = 'softmax') for i in range(n_task)]
    
    def call(self,x):
        #多个专家网络
        E_net = [expert(x) for expert in self.expert_layer]
        E_net = Concatenate(axis = 1)([e[:,tf.newaxis,:] for e in E_net]) #(bs,n_expert,n_dims)
        #多个门网络
        gate_net = [gate(x) for gate in self.gate_layers]     #n_task个(bs,n_expert)
        
        #每个towers等于，对应的门网络乘上所有的专家网络。
        towers = []
        for i in range(self.n_task):
            g = tf.expand_dims(gate_net[i],axis = -1)  #(bs,n_expert,1)
            _tower = tf.matmul(E_net, g,transpose_a=True)
            towers.append(Flatten()(_tower))           #(bs,expert_dim)
            
        return towers

def build_mmoe(sparse_cols,dense_cols,sparse_max_len,embed_dim,expert_dim,
              varlens_cols,varlens_max_len,n_expert,n_task,target = [],
              dnn_hidden_units = (64,),dnn_reg_l2 = 1e-5,drop_rate = 0.1,
                embedding_reg_l2 = 1e-6):
    
    
    #输入部分，分为sparse,varlens,dense部分。
    sparse_inputs = {f:Input([1],name = f) for f in sparse_cols}
    dense_inputs = {f:Input([1],name = f) for f in dense_cols}
    varlens_inputs = {f:Input([None,1],name = f) for f in varlens_cols}
        
    input_embed = {}
    #离散特征，embedding到k维
    for f in sparse_cols:
        _input = sparse_inputs[f]
        embedding = Embedding(sparse_max_len[f], embed_dim, 
            embeddings_regularizer=tf.keras.regularizers.l2(embedding_reg_l2)) 
        input_embed[f] =Flatten()(embedding(_input)) #(bs,k)
        
    #多标签离散变量
    for f in varlens_inputs:
        _input = varlens_inputs[f]
        mask = Masking(mask_value = 0).compute_mask(_input)
        embedding = Embedding(varlens_max_len[f], embed_dim,
            embeddings_regularizer=tf.keras.regularizers.l2(1e-6))
        _embed =Reshape([-1,embed_dim])(embedding(_input))
        out_embed = MeanPoolLayer(axis=1)(_embed,mask)
        input_embed[f] = out_embed
        
    input_embed.update(dense_inputs) #加入连续变量
    input_embed = Concatenate(axis = -1)([input_embed[f] for f in input_embed])
    for num in dnn_hidden_units:
        input_embed = Dropout(drop_rate)(Dense(num,activation = 'relu',
                    kernel_regularizer=regularizers.l2(dnn_reg_l2))(input_embed))
    
    #mmoe网络层
    towers = MmoeLayer(expert_dim,n_expert,n_task)(input_embed)
    outputs = [Dense(1,activation = 'sigmoid', kernel_regularizer=regularizers.l2(dnn_reg_l2),
                     name = f,use_bias = True)(_t) for _t,f in zip(towers,target)]
    inputs = [sparse_inputs[f] for f in sparse_inputs]+[varlens_inputs[f] for f in varlens_inputs]\
                +[dense_inputs[f] for f in dense_inputs]
    model = Model(inputs,outputs) 
    return model

In [64]:
# 构建模型，训练和评估
model = build_mmoe(sparse_features,dense_features,sparse_max_len,embed_dim = 64,expert_dim = 32,
          n_task = 3,n_expert = 6,varlens_cols = varlen_features,varlens_max_len = varlens_max_len,
          dnn_hidden_units = (64,128,64),target = target,dnn_reg_l2 = 0.001,drop_rate = 0.1)

adam = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(adam, loss = 'binary_crossentropy' ,metrics = ["accuracy"])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sex (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
race (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
risk (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
region (InputLayer)             [(None, 1)]          0                                            
____________________________________________________________________________________________

In [65]:
history = model.fit(train_model_input, train_labels,validation_data = (val_model_input,val_labels),
                    batch_size=1000, epochs=20, verbose=1)

Epoch 1/20
25/25 [==============================] - 11s 142ms/step - loss: 1869.4530 - VS_loss: 1081.5596 - linkage_loss: 227.5801 - retention_loss: 559.9071 - VS_accuracy: 0.5763 - linkage_accuracy: 0.6121 - retention_accuracy: 0.6503 - val_loss: 1751.6058 - val_VS_loss: 527.7377 - val_linkage_loss: 1061.1282 - val_retention_loss: 162.2075 - val_VS_accuracy: 0.3025 - val_linkage_accuracy: 0.6956 - val_retention_accuracy: 0.8419
Epoch 2/20
25/25 [==============================] - 1s 55ms/step - loss: 877.5034 - VS_loss: 397.8058 - linkage_loss: 127.1546 - retention_loss: 351.8724 - VS_accuracy: 0.5484 - linkage_accuracy: 0.5948 - retention_accuracy: 0.6381 - val_loss: 678.3790 - val_VS_loss: 332.4725 - val_linkage_loss: 317.8212 - val_retention_loss: 27.1996 - val_VS_accuracy: 0.6019 - val_linkage_accuracy: 0.5757 - val_retention_accuracy: 0.8381
Epoch 3/20
25/25 [==============================] - 3s 114ms/step - loss: 220.2439 - VS_loss: 162.8289 - linkage_loss: 25.5215 - retention_lo

In [66]:
pred = model.predict(test_model_input)
pred

[array([[0.70397633],
        [0.70397633],
        [0.70397633],
        ...,
        [0.70397633],
        [0.70397633],
        [0.70397633]], dtype=float32),
 array([[0.6985665],
        [0.6985665],
        [0.6985665],
        ...,
        [0.6985665],
        [0.6985665],
        [0.6985665]], dtype=float32),
 array([[0.83109313],
        [0.83109313],
        [0.83109313],
        ...,
        [0.83109313],
        [0.83109313],
        [0.83109313]], dtype=float32)]

In [67]:
print(accuracy_score(test_labels[0], (pred[0] > 0.5).astype(int)))
print(accuracy_score(test_labels[1], (pred[1] > 0.5).astype(int)))
print(accuracy_score(test_labels[2], (pred[2] > 0.5).astype(int)))

0.7768123138033763
0.7671300893743793
0.8259682224428997


In [68]:
print(roc_auc_score(test_labels[0], pred[0]))
print(roc_auc_score(test_labels[1], pred[1]))
print(roc_auc_score(test_labels[2], pred[2]))

0.5
0.5
0.5


### PLE

In [69]:
class MeanPoolLayer(Layer):
    def __init__(self, axis, **kwargs):
        super(MeanPoolLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, x, mask):
        mask = tf.expand_dims(tf.cast(mask,tf.float32),axis = -1)
        x = x * mask
        return K.sum(x, axis=self.axis) / (K.sum(mask, axis=self.axis) + 1e-9)

class PleLayer(tf.keras.layers.Layer):
    '''
    n_experts:list,每个任务使用几个expert。[2,3]第一个任务使用2个expert，第二个任务使用3个expert。
    n_expert_share:int,共享的部分设置的expert个数。
    expert_dim:int,每个专家网络输出的向量维度。
    n_task:int,任务个数。
    '''
    def __init__(self,n_task,n_experts,expert_dim,n_expert_share,dnn_reg_l2 = 1e-5):
        super(PleLayer, self).__init__()
        self.n_task = n_task
        
        # 生成多个任务特定网络和1个共享网络。
        self.E_layer = []
        for i in range(n_task):
            sub_exp = [Dense(expert_dim,activation = 'relu') for j in range(n_experts[i])]
            self.E_layer.append(sub_exp)
            
        self.share_layer = [Dense(expert_dim,activation = 'relu') for j in range(n_expert_share)]
        #定义门控网络
        self.gate_layers = [Dense(n_expert_share+n_experts[i],kernel_regularizer=regularizers.l2(dnn_reg_l2),
                                  activation = 'softmax') for i in range(n_task)]

    def call(self,x):
        #特定网络和共享网络
        E_net = [[expert(x) for expert in sub_expert] for sub_expert in self.E_layer]
        share_net = [expert(x) for expert in self.share_layer]
        
        #门的权重乘上，指定任务和共享任务的输出。
        towers = []
        for i in range(self.n_task):
            g = self.gate_layers[i](x)
            g = tf.expand_dims(g,axis = -1) #(bs,n_expert_share+n_experts[i],1)
            _e = share_net+E_net[i]  
            _e = Concatenate(axis = 1)([expert[:,tf.newaxis,:] for expert in _e]) #(bs,n_expert_share+n_experts[i],expert_dim)
            _tower = tf.matmul(_e, g,transpose_a=True)
            towers.append(Flatten()(_tower)) #(bs,expert_dim)
        return towers

def build_ple(sparse_cols,dense_cols,sparse_max_len,embed_dim,expert_dim = 4,
              varlens_cols = [],varlens_max_len = [],dnn_hidden_units = (64,64),
              n_task = 2,n_experts = [2,2],n_expert_share = 4,dnn_reg_l2 = 1e-6,
              drop_rate = 0.0,embedding_reg_l2 = 1e-6,targets = []):

   #输入部分，分为sparse,varlens,dense部分。
    sparse_inputs = {f:Input([1],name = f) for f in sparse_cols}
    dense_inputs = {f:Input([1],name = f) for f in dense_cols}
    varlens_inputs = {f:Input([None,1],name = f) for f in varlens_cols}
        
    input_embed = {}
    #离散特征，embedding到k维
    for f in sparse_cols:
        _input = sparse_inputs[f]
        embedding = Embedding(sparse_max_len[f], embed_dim, 
            embeddings_regularizer=tf.keras.regularizers.l2(embedding_reg_l2)) 
        input_embed[f] =Flatten()(embedding(_input)) #(bs,k)
        
    #多标签离散变量
    for f in varlens_inputs:
        _input = varlens_inputs[f]
        mask = Masking(mask_value = 0).compute_mask(_input)
        embedding = Embedding(varlens_max_len[f], embed_dim,
            embeddings_regularizer=tf.keras.regularizers.l2(1e-6))
        _embed =Reshape([-1,embed_dim])(embedding(_input))
        out_embed = MeanPoolLayer(axis=1)(_embed,mask)
        input_embed[f] = out_embed
        
    input_embed.update(dense_inputs) #加入连续变量
    input_embed = Concatenate(axis = -1)([input_embed[f] for f in input_embed])    
                                  
    for num in dnn_hidden_units:
        input_embed = Dropout(drop_rate)(Dense(num,activation = 'relu',
                    kernel_regularizer=regularizers.l2(dnn_reg_l2))(input_embed))
    #Ple网络层
    towers = PleLayer(n_task,n_experts,expert_dim,n_expert_share)(input_embed)
    outputs = [Dense(1,activation = 'sigmoid',kernel_regularizer=regularizers.l2(dnn_reg_l2),
                       name = f,use_bias = True)(_t) for f,_t in zip(targets,towers)]
    inputs = [sparse_inputs[f] for f in sparse_inputs]+[varlens_inputs[f] for f in varlens_inputs]\
                +[dense_inputs[f] for f in dense_inputs]
    model = Model(inputs,outputs) 
    return model

In [70]:
# 构建模型，训练和评估
model = build_ple(sparse_features,dense_features,sparse_max_len,embed_dim = 64,expert_dim = 16,
          varlens_cols = varlen_features,varlens_max_len = varlens_max_len,dnn_hidden_units = (64,64),
          n_task = 3,n_experts = [1,1,1],n_expert_share = 2,dnn_reg_l2 = 0.001,
          drop_rate = 0.1,embedding_reg_l2 = 0.001,targets = target)

adam = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(adam, loss = 'binary_crossentropy' ,metrics = ["accuracy"],)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sex (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
race (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
risk (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
region (InputLayer)             [(None, 1)]          0                                            
____________________________________________________________________________________________

In [71]:
history = model.fit(train_model_input, train_labels,validation_data = (val_model_input,val_labels),
                    batch_size=1000, epochs=20, verbose=1)

Epoch 1/20
25/25 [==============================] - 13s 163ms/step - loss: 2412.5435 - VS_loss: 744.0338 - linkage_loss: 781.3936 - retention_loss: 886.7078 - VS_accuracy: 0.5605 - linkage_accuracy: 0.6244 - retention_accuracy: 0.6687 - val_loss: 11565.8672 - val_VS_loss: 317.7469 - val_linkage_loss: 11246.2090 - val_retention_loss: 1.2507 - val_VS_accuracy: 0.3086 - val_linkage_accuracy: 0.6864 - val_retention_accuracy: 0.8419
Epoch 2/20
25/25 [==============================] - 1s 60ms/step - loss: 574.3810 - VS_loss: 217.2601 - linkage_loss: 353.6669 - retention_loss: 2.3894 - VS_accuracy: 0.5381 - linkage_accuracy: 0.5647 - retention_accuracy: 0.8091 - val_loss: 1132.9025 - val_VS_loss: 1130.0895 - val_linkage_loss: 0.7963 - val_retention_loss: 0.5989 - val_VS_accuracy: 0.7367 - val_linkage_accuracy: 0.3306 - val_retention_accuracy: 0.8419
Epoch 3/20
25/25 [==============================] - 2s 61ms/step - loss: 32.0332 - VS_loss: 27.0306 - linkage_loss: 2.9339 - retention_loss: 0.74

In [72]:
pred = model.predict(test_model_input)
pred

[array([[0.70516324],
        [0.70516324],
        [0.70516324],
        ...,
        [0.70516324],
        [0.70516324],
        [0.70516324]], dtype=float32),
 array([[0.699403],
        [0.699403],
        [0.699403],
        ...,
        [0.699403],
        [0.699403],
        [0.699403]], dtype=float32),
 array([[0.83072495],
        [0.83072495],
        [0.83072495],
        ...,
        [0.83072495],
        [0.83072495],
        [0.83072495]], dtype=float32)]

In [73]:
print(accuracy_score(test_labels[0], (pred[0] > 0.5).astype(int)))
print(accuracy_score(test_labels[1], (pred[1] > 0.5).astype(int)))
print(accuracy_score(test_labels[2], (pred[2] > 0.5).astype(int)))

0.7768123138033763
0.7671300893743793
0.8259682224428997


In [74]:
print(roc_auc_score(test_labels[0], pred[0]))
print(roc_auc_score(test_labels[1], pred[1]))
print(roc_auc_score(test_labels[2], pred[2]))

0.5
0.5
0.5


## Model 3

In [41]:
temp = variables[variables['model3'] != 'delete']
data = data_origin[temp['variables'].tolist()]
data

,TIME_DXDATE_TESTDATE,time_window_index_year,VL_interpretation,VL,VS,VR,VB,Months_to_ini_VS,VR_N,VR_size,...,msld.cum_1,metacanc.cum_1,aids.cum_1,Depression_1,Anxiety_1,Psychiatric_disorder_1,Alcohol_use_1,Tobacco_use_1,Illicit_drug_use_1,visits_1
0,591.000000,2,=,2902.0,0,0,0,69.075000,0,none,...,0.0,0.0,0.0,0,0,0,0,0,0,4
1,1018.000000,3,=,4804.0,0,0,0,69.075000,0,none,...,0.0,0.0,0.0,0,1,0,1,1,1,2
2,2072.250000,6,<,200.0,1,0,0,69.075000,0,none,...,0.0,0.0,0.0,0,0,0,0,0,0,2
3,2495.333333,7,<,200.0,1,0,0,69.075000,0,none,...,0.0,0.0,0.0,0,0,0,0,0,0,8
4,2793.333333,8,<,200.0,1,0,0,69.075000,0,none,...,0.0,0.0,0.0,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40274,2737.500000,8,<,200.0,1,0,0,17.616667,0,none,...,0.0,0.0,0.0,0,0,0,0,0,0,3
40275,3193.000000,9,<,200.0,1,0,0,17.616667,0,none,...,0.0,0.0,0.0,0,0,0,0,0,0,8
40276,3499.666667,10,<,200.0,1,0,0,17.616667,0,none,...,0.0,0.0,0.0,0,0,0,0,0,0,3
40277,3884.000000,11,<,200.0,1,0,0,17.616667,0,none,...,0.0,0.0,0.0,0,0,0,0,0,0,6


In [42]:
target = variables[variables['model3'] == 'outcome']['variables'].tolist()
sparse_features = variables[variables['model3'] == 'cat']['variables'].tolist()
dense_features = variables[variables['model3'] == 'num']['variables'].tolist()
varlen_features = []

In [43]:
encoder = {}
# 稀疏特征编码
for featid in sparse_features:
    # print(f"编码ID字段：{featid}")
    encoder[featid] = {uid:ucode+1 for ucode,uid in enumerate(data[featid].unique())} 
    data[featid] = data[featid].apply(lambda x: encoder[featid].get(x,0))
    
# 生成输入特征设置
sparse_max_len = {f:len(encoder[f]) + 1 for f in sparse_features}
varlens_max_len = {f:len(encoder[f]) + 1 for f in varlen_features}
feature_names = sparse_features+varlen_features+dense_features

C:\Users\Yunqing\AppData\Local\Temp\ipykernel_46932\2408031544.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[featid] = data[featid].apply(lambda x: encoder[featid].get(x,0))


In [44]:
n_train = round(data.shape[0] * 0.6)
n_val = round(data.shape[0] * 0.2)

train = data[:n_train]
val = data[n_train:(n_train+n_val)]
test = data[(n_train+n_val):]

In [45]:
# 构建输入数据
train_model_input = {name: train[name] if name not in varlen_features else np.stack(train[name]) for name in feature_names } #训练模型的输入，字典类型。名称和具体值
val_model_input = {name: val[name] if name not in varlen_features else np.stack(val[name]) for name in feature_names }
test_model_input = {name: test[name] if name not in varlen_features else np.stack(test[name]) for name in feature_names}

train_labels = [train[y].values for y in target]
val_labels = [val[y].values for y in target]
test_labels = [test[y].values for y in target]

### Seperate models

#### VS

In [81]:
X_train = data[:(n_train+n_val)].copy()
X_train.drop(columns=target, inplace=True)
y_train = data[:(n_train+n_val)]["VS"]
X_train

,TIME_DXDATE_TESTDATE,time_window_index_year,VL_interpretation,VL,VR,VB,Months_to_ini_VS,VR_N,VR_size,prop_time,...,msld.cum_1,metacanc.cum_1,aids.cum_1,Depression_1,Anxiety_1,Psychiatric_disorder_1,Alcohol_use_1,Tobacco_use_1,Illicit_drug_use_1,visits_1
0,591.000000,2,1,2902.0,1,1,69.075000,0,1,0.000000,...,0.0,0.0,0.0,1,1,1,1,1,1,4
1,1018.000000,3,1,4804.0,1,1,69.075000,0,1,0.000000,...,0.0,0.0,0.0,1,2,1,2,2,2,2
2,2072.250000,6,2,200.0,1,1,69.075000,0,1,0.151356,...,0.0,0.0,0.0,1,1,1,1,1,1,2
3,2495.333333,7,2,200.0,1,1,69.075000,0,1,0.267065,...,0.0,0.0,0.0,1,1,1,1,1,1,8
4,2793.333333,8,2,200.0,1,1,69.075000,0,1,0.335538,...,0.0,0.0,0.0,1,1,1,1,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32218,1693.000000,5,2,200.0,1,1,18.966667,0,1,0.799523,...,0.0,0.0,0.0,1,1,1,1,1,1,4
32219,1964.000000,6,2,200.0,1,1,18.966667,0,1,0.831713,...,0.0,0.0,0.0,1,1,1,1,1,1,2
32220,2392.500000,7,2,200.0,1,1,18.966667,0,1,0.858014,...,0.0,0.0,0.0,1,1,1,1,1,1,6
32221,2771.000000,8,1,200.0,1,2,18.966667,0,1,0.873443,...,0.0,0.0,0.0,1,1,1,1,1,1,5


In [82]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 64)                7808      
_________________________________________________________________
dense_66 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_67 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_68 (Dense)             (None, 1)                 65        
Total params: 24,449
Trainable params: 24,449
Non-trainable params: 0
_________________________________________________________________


In [83]:
# Train the model
history = model.fit(X_train.to_numpy(), np.array(y_train), epochs=20, batch_size=1000, validation_split=0.2)

Epoch 1/20
26/26 [==============================] - 1s 15ms/step - loss: 3267.5876 - accuracy: 0.6166 - val_loss: 2002.6642 - val_accuracy: 0.5922
Epoch 2/20
26/26 [==============================] - 0s 6ms/step - loss: 2391.6963 - accuracy: 0.6438 - val_loss: 23766.2559 - val_accuracy: 0.8272
Epoch 3/20
26/26 [==============================] - 0s 6ms/step - loss: 3555.5254 - accuracy: 0.7206 - val_loss: 7559.5059 - val_accuracy: 0.8493
Epoch 4/20
26/26 [==============================] - 0s 7ms/step - loss: 1261.6584 - accuracy: 0.7174 - val_loss: 3951.4124 - val_accuracy: 0.8470
Epoch 5/20
26/26 [==============================] - 0s 7ms/step - loss: 2171.7959 - accuracy: 0.7202 - val_loss: 5442.8867 - val_accuracy: 0.8507
Epoch 6/20
26/26 [==============================] - 0s 6ms/step - loss: 611.2400 - accuracy: 0.7538 - val_loss: 4492.0742 - val_accuracy: 0.8574
Epoch 7/20
26/26 [==============================] - 0s 10ms/step - loss: 1361.1686 - accuracy: 0.7821 - val_loss: 320.4504 

In [84]:
X_test = test.copy()
X_test.drop(columns=target, inplace=True)
pred = model.predict(X_test)
pred

array([[1.       ],
       [1.       ],
       [1.       ],
       ...,
       [1.       ],
       [1.       ],
       [0.9537422]], dtype=float32)

In [85]:
print(accuracy_score(test_labels[0], (pred > 0.5).astype(int)))

0.6904170804369414


In [86]:
print(roc_auc_score(test_labels[0], pred))

0.7117912875745964


#### RIC

In [87]:
X_train = data[:(n_train+n_val)].copy()
X_train.drop(columns=target, inplace=True)
y_train = data[:(n_train+n_val)]["retention"]

In [88]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')   # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_69 (Dense)             (None, 64)                7808      
_________________________________________________________________
dense_70 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_71 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 65        
Total params: 24,449
Trainable params: 24,449
Non-trainable params: 0
_________________________________________________________________


In [89]:
# Train the model
history = model.fit(X_train.to_numpy(), np.array(y_train), epochs=20, batch_size=1000, validation_split=0.2)

Epoch 1/20
26/26 [==============================] - 4s 122ms/step - loss: 2749.2988 - accuracy: 0.6610 - val_loss: 4585.3027 - val_accuracy: 0.5493
Epoch 2/20
26/26 [==============================] - 0s 5ms/step - loss: 1011.2532 - accuracy: 0.6524 - val_loss: 2155.1438 - val_accuracy: 0.6109
Epoch 3/20
26/26 [==============================] - 0s 5ms/step - loss: 457.7648 - accuracy: 0.7368 - val_loss: 8436.9385 - val_accuracy: 0.3462
Epoch 4/20
26/26 [==============================] - 0s 5ms/step - loss: 1024.1942 - accuracy: 0.6631 - val_loss: 569.9102 - val_accuracy: 0.8369
Epoch 5/20
26/26 [==============================] - 0s 5ms/step - loss: 1030.1630 - accuracy: 0.6461 - val_loss: 1246.7502 - val_accuracy: 0.7286
Epoch 6/20
26/26 [==============================] - 0s 5ms/step - loss: 609.2635 - accuracy: 0.7372 - val_loss: 470.0819 - val_accuracy: 0.7784
Epoch 7/20
26/26 [==============================] - 3s 116ms/step - loss: 1119.7949 - accuracy: 0.6565 - val_loss: 7122.7812 -

In [90]:
X_test = test.copy()
X_test.drop(columns=target, inplace=True)
pred = model.predict(X_test)
pred

array([[1.       ],
       [1.       ],
       [1.       ],
       ...,
       [0.9999012],
       [0.9999926],
       [0.9976463]], dtype=float32)

In [91]:
print(accuracy_score(test_labels[1], (pred > 0.5).astype(int)))

0.7329940417080437


In [92]:
print(roc_auc_score(test_labels[1], pred))

0.5230714061454171


#### LTC

In [93]:
X_train = data[:(n_train+n_val)].copy()
X_train.drop(columns=target, inplace=True)
y_train = data[:(n_train+n_val)]["linkage"]

In [94]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')   # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 64)                7808      
_________________________________________________________________
dense_74 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_75 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_76 (Dense)             (None, 1)                 65        
Total params: 24,449
Trainable params: 24,449
Non-trainable params: 0
_________________________________________________________________


In [95]:
# Train the model
history = model.fit(X_train.to_numpy(), np.array(y_train), epochs=20, batch_size=1000, validation_split=0.2)

Epoch 1/20
26/26 [==============================] - 1s 13ms/step - loss: 2130.4707 - accuracy: 0.6079 - val_loss: 15420.1895 - val_accuracy: 0.6925
Epoch 2/20
26/26 [==============================] - 0s 6ms/step - loss: 668.9637 - accuracy: 0.6012 - val_loss: 37732.8477 - val_accuracy: 0.6926
Epoch 3/20
26/26 [==============================] - 0s 6ms/step - loss: 1357.0073 - accuracy: 0.6006 - val_loss: 9085.8936 - val_accuracy: 0.6917
Epoch 4/20
26/26 [==============================] - 0s 6ms/step - loss: 1349.6593 - accuracy: 0.6477 - val_loss: 28684.1836 - val_accuracy: 0.6337
Epoch 5/20
26/26 [==============================] - 0s 6ms/step - loss: 469.9576 - accuracy: 0.5670 - val_loss: 9812.9795 - val_accuracy: 0.6853
Epoch 6/20
26/26 [==============================] - 0s 7ms/step - loss: 976.9426 - accuracy: 0.6430 - val_loss: 21349.7402 - val_accuracy: 0.6379
Epoch 7/20
26/26 [==============================] - 0s 10ms/step - loss: 527.9663 - accuracy: 0.5707 - val_loss: 5686.5347

In [96]:
X_test = test.copy()
X_test.drop(columns=target, inplace=True)
pred = model.predict(X_test)
pred

array([[1.        ],
       [1.        ],
       [1.        ],
       ...,
       [0.41003317],
       [0.75006294],
       [0.98746324]], dtype=float32)

In [97]:
print(accuracy_score(test_labels[1], (pred > 0.5).astype(int)))

0.7158639523336644


In [98]:
print(roc_auc_score(test_labels[1], pred))

0.5307445522043044


### New MMoE

In [49]:
train_labels[1]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [54]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy="auto", random_state=42)

# 分别对 VS, VB, VR 进行 oversampling
X_resampled_vs, y_resampled_vs = ros.fit_resample(pd.DataFrame(train_model_input).reset_index(drop=True), train_labels[0])
X_resampled_vb, y_resampled_vb = ros.fit_resample(pd.DataFrame(train_model_input).reset_index(drop=True), train_labels[1])
X_resampled_vr, y_resampled_vr = ros.fit_resample(pd.DataFrame(train_model_input).reset_index(drop=True), train_labels[2])

# 取交集，确保 `X_resampled` 统一（使用 .loc 代替 .iloc）
X_resampled = X_resampled_vs.reset_index(drop=True)

# 重新组合 y
y_resampled_combined = [y_resampled_vs, y_resampled_vb, y_resampled_vr]

### MMoE

In [58]:
class MeanPoolLayer(Layer):
    def __init__(self, axis, **kwargs):
        super(MeanPoolLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, x, mask):
        mask = tf.expand_dims(tf.cast(mask,tf.float32),axis = -1)
        x = x * mask
        return K.sum(x, axis=self.axis) / (K.sum(mask, axis=self.axis) + 1e-9)

class MmoeLayer(tf.keras.layers.Layer):
    def __init__(self,expert_dim,n_expert,n_task):
        super(MmoeLayer, self).__init__()
        self.n_task = n_task
        self.expert_layer = [Dense(expert_dim,activation = 'relu') for i in range(n_expert)]
        self.gate_layers = [Dense(n_expert,activation = 'softmax') for i in range(n_task)]
    
    def call(self,x):
        #多个专家网络
        E_net = [expert(x) for expert in self.expert_layer]
        E_net = Concatenate(axis = 1)([e[:,tf.newaxis,:] for e in E_net]) #(bs,n_expert,n_dims)
        #多个门网络
        gate_net = [gate(x) for gate in self.gate_layers]     #n_task个(bs,n_expert)
        
        #每个towers等于，对应的门网络乘上所有的专家网络。
        towers = []
        for i in range(self.n_task):
            g = tf.expand_dims(gate_net[i],axis = -1)  #(bs,n_expert,1)
            _tower = tf.matmul(E_net, g,transpose_a=True)
            towers.append(Flatten()(_tower))           #(bs,expert_dim)
            
        return towers

def build_mmoe(sparse_cols,dense_cols,sparse_max_len,embed_dim,expert_dim,
              varlens_cols,varlens_max_len,n_expert,n_task,target = [],
              dnn_hidden_units = (64,),dnn_reg_l2 = 1e-5,drop_rate = 0.1,
                embedding_reg_l2 = 1e-6):
    
    
    #输入部分，分为sparse,varlens,dense部分。
    sparse_inputs = {f:Input([1],name = f) for f in sparse_cols}
    dense_inputs = {f:Input([1],name = f) for f in dense_cols}
    varlens_inputs = {f:Input([None,1],name = f) for f in varlens_cols}
        
    input_embed = {}
    #离散特征，embedding到k维
    for f in sparse_cols:
        _input = sparse_inputs[f]
        embedding = Embedding(sparse_max_len[f], embed_dim, 
            embeddings_regularizer=tf.keras.regularizers.l2(embedding_reg_l2)) 
        input_embed[f] =Flatten()(embedding(_input)) #(bs,k)
        
    #多标签离散变量
    for f in varlens_inputs:
        _input = varlens_inputs[f]
        mask = Masking(mask_value = 0).compute_mask(_input)
        embedding = Embedding(varlens_max_len[f], embed_dim,
            embeddings_regularizer=tf.keras.regularizers.l2(1e-6))
        _embed =Reshape([-1,embed_dim])(embedding(_input))
        out_embed = MeanPoolLayer(axis=1)(_embed,mask)
        input_embed[f] = out_embed
        
    input_embed.update(dense_inputs) #加入连续变量
    input_embed = Concatenate(axis = -1)([input_embed[f] for f in input_embed])
    for num in dnn_hidden_units:
        input_embed = Dropout(drop_rate)(Dense(num,activation = 'relu',
                    kernel_regularizer=regularizers.l2(dnn_reg_l2))(input_embed))
    
    #mmoe网络层
    towers = MmoeLayer(expert_dim,n_expert,n_task)(input_embed)
    outputs = [Dense(1,activation = 'sigmoid', kernel_regularizer=regularizers.l2(dnn_reg_l2),
                     name = f,use_bias = True)(_t) for _t,f in zip(towers,target)]
    inputs = [sparse_inputs[f] for f in sparse_inputs]+[varlens_inputs[f] for f in varlens_inputs]\
                +[dense_inputs[f] for f in dense_inputs]
    model = Model(inputs,outputs) 
    return model

In [59]:
model = build_mmoe(
    sparse_features, dense_features, sparse_max_len, embed_dim=64, expert_dim=32,
    n_task=3, n_expert=6, varlens_cols=varlen_features, varlens_max_len=varlens_max_len,
    dnn_hidden_units=(64, 128, 64), target=target, dnn_reg_l2=0.001, drop_rate=0.1
)

adam = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=["AUC"])  # AUC 适用于不平衡数据

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
VL_interpretation (InputLayer)  [(None, 1)]          0                                            
__________________________________________________________________________________________________
VR (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
VB (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
VR_size (InputLayer)            [(None, 1)]          0                                            
____________________________________________________________________________________________

In [61]:
# **确保 X_resampled 经过 Oversampling**
train_model_input_resampled = {
    name: X_resampled[name].values if name not in varlen_features 
    else np.stack(X_resampled[name].values) for name in feature_names
}

In [63]:
# **如果 y_resampled_combined 是 DataFrame，先转换成 numpy**
if isinstance(y_resampled_combined, np.ndarray):
    train_labels_resampled = [
        y_resampled_combined[:, 0],  # VS 任务
        y_resampled_combined[:, 1],  # VB 任务
        y_resampled_combined[:, 2]   # VR 任务
    ]
elif isinstance(y_resampled_combined, pd.DataFrame):
    train_labels_resampled = [
        y_resampled_combined.iloc[:, 0].values,
        y_resampled_combined.iloc[:, 1].values,
        y_resampled_combined.iloc[:, 2].values
    ]
else:
    raise ValueError("y_resampled_combined format not supported!")

ValueError: y_resampled_combined format not supported!

In [62]:
history = model.fit(
    train_model_input_resampled,  # 用 Oversampled 数据
    train_labels_resampled,  # 用 Oversampled 标签
    validation_data=(val_model_input, val_labels), 
    batch_size=1000, epochs=20, verbose=1)

NameError: name 'train_labels_resampled' is not defined

In [102]:
pred = model.predict(test_model_input)
pred

[array([[0.7047897],
        [0.7047897],
        [0.7047897],
        ...,
        [0.7047897],
        [0.7047897],
        [0.7047897]], dtype=float32),
 array([[0.6987062],
        [0.6987062],
        [0.6987062],
        ...,
        [0.6987062],
        [0.6987062],
        [0.6987062]], dtype=float32),
 array([[0.8310056],
        [0.8310056],
        [0.8310056],
        ...,
        [0.8310056],
        [0.8310056],
        [0.8310056]], dtype=float32)]

In [103]:
print(accuracy_score(test_labels[0], (pred[0] > 0.5).astype(int)))
print(accuracy_score(test_labels[1], (pred[1] > 0.5).astype(int)))
print(accuracy_score(test_labels[2], (pred[2] > 0.5).astype(int)))

0.7768123138033763
0.7671300893743793
0.8259682224428997


In [104]:
print(roc_auc_score(test_labels[0], pred[0]))
print(roc_auc_score(test_labels[1], pred[1]))
print(roc_auc_score(test_labels[2], pred[2]))

0.5
0.5
0.5


### PLE

In [105]:
class MeanPoolLayer(Layer):
    def __init__(self, axis, **kwargs):
        super(MeanPoolLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, x, mask):
        mask = tf.expand_dims(tf.cast(mask,tf.float32),axis = -1)
        x = x * mask
        return K.sum(x, axis=self.axis) / (K.sum(mask, axis=self.axis) + 1e-9)

class PleLayer(tf.keras.layers.Layer):
    '''
    n_experts:list,每个任务使用几个expert。[2,3]第一个任务使用2个expert，第二个任务使用3个expert。
    n_expert_share:int,共享的部分设置的expert个数。
    expert_dim:int,每个专家网络输出的向量维度。
    n_task:int,任务个数。
    '''
    def __init__(self,n_task,n_experts,expert_dim,n_expert_share,dnn_reg_l2 = 1e-5):
        super(PleLayer, self).__init__()
        self.n_task = n_task
        
        # 生成多个任务特定网络和1个共享网络。
        self.E_layer = []
        for i in range(n_task):
            sub_exp = [Dense(expert_dim,activation = 'relu') for j in range(n_experts[i])]
            self.E_layer.append(sub_exp)
            
        self.share_layer = [Dense(expert_dim,activation = 'relu') for j in range(n_expert_share)]
        #定义门控网络
        self.gate_layers = [Dense(n_expert_share+n_experts[i],kernel_regularizer=regularizers.l2(dnn_reg_l2),
                                  activation = 'softmax') for i in range(n_task)]

    def call(self,x):
        #特定网络和共享网络
        E_net = [[expert(x) for expert in sub_expert] for sub_expert in self.E_layer]
        share_net = [expert(x) for expert in self.share_layer]
        
        #门的权重乘上，指定任务和共享任务的输出。
        towers = []
        for i in range(self.n_task):
            g = self.gate_layers[i](x)
            g = tf.expand_dims(g,axis = -1) #(bs,n_expert_share+n_experts[i],1)
            _e = share_net+E_net[i]  
            _e = Concatenate(axis = 1)([expert[:,tf.newaxis,:] for expert in _e]) #(bs,n_expert_share+n_experts[i],expert_dim)
            _tower = tf.matmul(_e, g,transpose_a=True)
            towers.append(Flatten()(_tower)) #(bs,expert_dim)
        return towers

def build_ple(sparse_cols,dense_cols,sparse_max_len,embed_dim,expert_dim = 4,
              varlens_cols = [],varlens_max_len = [],dnn_hidden_units = (64,64),
              n_task = 2,n_experts = [2,2],n_expert_share = 4,dnn_reg_l2 = 1e-6,
              drop_rate = 0.0,embedding_reg_l2 = 1e-6,targets = []):

   #输入部分，分为sparse,varlens,dense部分。
    sparse_inputs = {f:Input([1],name = f) for f in sparse_cols}
    dense_inputs = {f:Input([1],name = f) for f in dense_cols}
    varlens_inputs = {f:Input([None,1],name = f) for f in varlens_cols}
        
    input_embed = {}
    #离散特征，embedding到k维
    for f in sparse_cols:
        _input = sparse_inputs[f]
        embedding = Embedding(sparse_max_len[f], embed_dim, 
            embeddings_regularizer=tf.keras.regularizers.l2(embedding_reg_l2)) 
        input_embed[f] =Flatten()(embedding(_input)) #(bs,k)
        
    #多标签离散变量
    for f in varlens_inputs:
        _input = varlens_inputs[f]
        mask = Masking(mask_value = 0).compute_mask(_input)
        embedding = Embedding(varlens_max_len[f], embed_dim,
            embeddings_regularizer=tf.keras.regularizers.l2(1e-6))
        _embed =Reshape([-1,embed_dim])(embedding(_input))
        out_embed = MeanPoolLayer(axis=1)(_embed,mask)
        input_embed[f] = out_embed
        
    input_embed.update(dense_inputs) #加入连续变量
    input_embed = Concatenate(axis = -1)([input_embed[f] for f in input_embed])    
                                  
    for num in dnn_hidden_units:
        input_embed = Dropout(drop_rate)(Dense(num,activation = 'relu',
                    kernel_regularizer=regularizers.l2(dnn_reg_l2))(input_embed))
    #Ple网络层
    towers = PleLayer(n_task,n_experts,expert_dim,n_expert_share)(input_embed)
    outputs = [Dense(1,activation = 'sigmoid',kernel_regularizer=regularizers.l2(dnn_reg_l2),
                       name = f,use_bias = True)(_t) for f,_t in zip(targets,towers)]
    inputs = [sparse_inputs[f] for f in sparse_inputs]+[varlens_inputs[f] for f in varlens_inputs]\
                +[dense_inputs[f] for f in dense_inputs]
    model = Model(inputs,outputs) 
    return model

In [106]:
# 构建模型，训练和评估
model = build_ple(sparse_features,dense_features,sparse_max_len,embed_dim = 64,expert_dim = 16,
          varlens_cols = varlen_features,varlens_max_len = varlens_max_len,dnn_hidden_units = (64,64),
          n_task = 3,n_experts = [1,1,1],n_expert_share = 2,dnn_reg_l2 = 0.001,
          drop_rate = 0.1,embedding_reg_l2 = 0.001,targets = target)

adam = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(adam, loss = 'binary_crossentropy' ,metrics = ["accuracy"],)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
VL_interpretation (InputLayer)  [(None, 1)]          0                                            
__________________________________________________________________________________________________
VR (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
VB (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
VR_size (InputLayer)            [(None, 1)]          0                                            
____________________________________________________________________________________________

In [107]:
history = model.fit(train_model_input, train_labels,validation_data = (val_model_input,val_labels),
                    batch_size=1000, epochs=25, verbose=1)

Epoch 1/25
25/25 [==============================] - 13s 166ms/step - loss: 1871.6349 - VS_loss: 1003.0266 - linkage_loss: 249.8846 - retention_loss: 618.4240 - VS_accuracy: 0.6005 - linkage_accuracy: 0.5925 - retention_accuracy: 0.6945 - val_loss: 2194.8425 - val_VS_loss: 435.8506 - val_linkage_loss: 1372.0815 - val_retention_loss: 386.5444 - val_VS_accuracy: 0.8035 - val_linkage_accuracy: 0.6878 - val_retention_accuracy: 0.8189
Epoch 2/25
25/25 [==============================] - 2s 69ms/step - loss: 822.6893 - VS_loss: 235.4770 - linkage_loss: 54.2323 - retention_loss: 532.5888 - VS_accuracy: 0.6716 - linkage_accuracy: 0.6028 - retention_accuracy: 0.6365 - val_loss: 207.5065 - val_VS_loss: 114.1879 - val_linkage_loss: 26.4861 - val_retention_loss: 66.4294 - val_VS_accuracy: 0.7099 - val_linkage_accuracy: 0.6513 - val_retention_accuracy: 0.8347
Epoch 3/25
25/25 [==============================] - 2s 68ms/step - loss: 128.3707 - VS_loss: 90.0404 - linkage_loss: 9.6802 - retention_loss: 2

Epoch 21/25
25/25 [==============================] - 2s 67ms/step - loss: 1.6382 - VS_loss: 0.4578 - linkage_loss: 0.6191 - retention_loss: 0.4883 - VS_accuracy: 0.8229 - linkage_accuracy: 0.6993 - retention_accuracy: 0.8311 - val_loss: 1.5485 - val_VS_loss: 0.4245 - val_linkage_loss: 0.6138 - val_retention_loss: 0.4374 - val_VS_accuracy: 0.8416 - val_linkage_accuracy: 0.6964 - val_retention_accuracy: 0.8419
Epoch 22/25
25/25 [==============================] - 2s 70ms/step - loss: 1.6374 - VS_loss: 0.4408 - linkage_loss: 0.6150 - retention_loss: 0.5089 - VS_accuracy: 0.8261 - linkage_accuracy: 0.6992 - retention_accuracy: 0.8311 - val_loss: 1.5446 - val_VS_loss: 0.4212 - val_linkage_loss: 0.6137 - val_retention_loss: 0.4373 - val_VS_accuracy: 0.8431 - val_linkage_accuracy: 0.6964 - val_retention_accuracy: 0.8419
Epoch 23/25
25/25 [==============================] - 2s 67ms/step - loss: 1.5841 - VS_loss: 0.4355 - linkage_loss: 0.6130 - retention_loss: 0.4632 - VS_accuracy: 0.8314 - linka

In [108]:
pred = model.predict(test_model_input)
pred

[array([[0.7822732 ],
        [0.78229797],
        [0.84526813],
        ...,
        [0.84526813],
        [0.84526813],
        [0.827006  ]], dtype=float32),
 array([[0.7036179],
        [0.7036938],
        [0.693956 ],
        ...,
        [0.693956 ],
        [0.693956 ],
        [0.697632 ]], dtype=float32),
 array([[0.81969744],
        [0.81991976],
        [0.84267193],
        ...,
        [0.84267193],
        [0.84267193],
        [0.83497804]], dtype=float32)]

In [109]:
print(accuracy_score(test_labels[0], (pred[0] > 0.5).astype(int)))
print(accuracy_score(test_labels[1], (pred[1] > 0.5).astype(int)))
print(accuracy_score(test_labels[2], (pred[2] > 0.5).astype(int)))

0.8799652432969215
0.7671300893743793
0.8259682224428997


In [110]:
print(roc_auc_score(test_labels[0], pred[0]))
print(roc_auc_score(test_labels[1], pred[1]))
print(roc_auc_score(test_labels[2], pred[2]))

0.7557411274414134
0.5263010105505759
0.49108132484530875
